In [2]:
import numpy as np
import pandas as pd

In [3]:
df1 = pd.read_excel("sales_data.xlsx")
df1.head(5)

,transaction_id,date,customer_id,category,product,quantity,unit_price,total_price,total_cost,gross_profit,profit_margin,payment_method,in_stock
0,TXN000000,2023-01-01 00:01:00,CUST14629,Prescription,Amoxicillin 25mg,1,23.41,23.41,8.99,14.42,61.60,Debit Card,75
1,TXN000001,2023-01-01 00:01:00,CUST38699,Prescription,Jardiance 25mg,3,43.84,131.52,50.97,80.55,61.25,Credit Card,148
2,TXN000002,2023-01-01 00:02:00,CUST33119,Prescription,Atorvastatin 10mg,1,26.02,26.02,12.99,13.03,50.08,Debit Card,194
3,TXN000003,2023-01-01 00:02:00,CUST14447,Prescription,Metoprolol 50mg,3,16.58,49.74,23.97,25.77,51.81,Insurance,154
4,TXN000004,2023-01-01 00:02:00,CUST45754,Prescription,Cyclobenzaprine 25mg,1,19.06,19.06,6.99,12.07,63.33,Debit Card,148


In [4]:
df2 = df1.drop(['transaction_id',	'customer_id', 'category','profit_margin', 'payment_method', 'in_stock'], axis='columns')
df2['Year'] = df2['date'].dt.year
df2.head()

,date,product,quantity,unit_price,total_price,total_cost,gross_profit,Year
0,2023-01-01 00:01:00,Amoxicillin 25mg,1,23.41,23.41,8.99,14.42,2023
1,2023-01-01 00:01:00,Jardiance 25mg,3,43.84,131.52,50.97,80.55,2023
2,2023-01-01 00:02:00,Atorvastatin 10mg,1,26.02,26.02,12.99,13.03,2023
3,2023-01-01 00:02:00,Metoprolol 50mg,3,16.58,49.74,23.97,25.77,2023
4,2023-01-01 00:02:00,Cyclobenzaprine 25mg,1,19.06,19.06,6.99,12.07,2023


In [5]:
df2.describe()

,date,quantity,unit_price,total_price,total_cost,gross_profit,Year
count,500000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.0
mean,2023-07-02 14:28:34.357679360,1.999144,19.316783,38.612645,15.615808,22.996837,2023.0
min,2023-01-01 00:01:00,1.000000,2.990000,2.990000,1.490000,1.500000,2023.0
25%,2023-04-02 06:35:00,1.000000,13.420000,20.020000,7.990000,11.880000,2023.0
50%,2023-07-02 20:05:30,2.000000,18.060000,33.260000,13.470000,19.480000,2023.0
75%,2023-10-01 16:54:15,3.000000,23.400000,51.000000,20.970000,30.340000,2023.0
max,2023-12-31 23:59:00,3.000000,59.990000,179.970000,53.970000,126.000000,2023.0
std,NaN,0.816434,8.646752,24.448307,10.079391,14.927038,0.0


In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

X = df2[['product', 'quantity', 'unit_price','total_cost','total_price','Year']]
y = df2['gross_profit']

categorical_cols = ['product']
numerical_cols = ['quantity', 'unit_price','total_cost','total_price','Year']

column_transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), categorical_cols)], remainder='passthrough')

X = column_transformer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dt_model = DecisionTreeRegressor()
dt_model.fit(X_train, y_train)
dt_model.score(X_test, y_test)

0.9999998361139301

In [13]:
new_data = {
    'product': 'Facial Cleanser (CeraVe)',
    'quantity': 6,
    'unit_price': 56.41,
    'total_cost': 90.67,
    'total_price': 23.41,
    'Year': 2024
}
new_df = pd.DataFrame([new_data])
new_data_transformed = column_transformer.transform(new_df)
pred = dt_model.predict(new_data_transformed)
print(pred[0])

11.0


In [8]:
import pickle
pickle.dump(dt_model, open('project_model.pkl', 'wb'))

In [9]:
pickle.dump(column_transformer, open('column_transformer.pkl', 'wb'))

In [10]:
product_list = df1['product'].drop_duplicates().sort_values().tolist()
import json
# Save the list to a JSON file
with open('product_list.json', 'w') as json_file:
    json.dump(product_list, json_file)